In [80]:
import librosa
song_name = 'A'

In [81]:
y, sr = librosa.load('StepMania/Songs/StepMania 5/{0}/{0}.mp3'.format(song_name))

In [82]:
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

Estimated tempo: 184.57 beats per minute


In [83]:
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

In [84]:
len(y)

2845440

In [85]:
def write_song_header(output_stepfile):
    keys = ['VERSION', 'TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH']
    header_info = {
        'VERSION': 0.82,
        'TITLE': song_name,
        'MUSIC': '{0}.mp3'.format(song_name),
        'OFFSET': -0.090,
        'SAMPLESTART': 105.760,
        'SAMPLELENGTH': 15
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)

In [86]:
def write_step_header(output_stepfile):
    print("//---------------dance-single - ----------------", file=output_stepfile)
    keys = ['NOTEDATA', 'CHARTNAME', 'STEPSTYPE', 'DIFFICULTY', 'METER', 'RADARVALUES', 'BPMS']
    step_info = {
        'NOTEDATA': '',
        'CHARTNAME': 'Kommisar',
        'STEPSTYPE': 'dance-single',
        'DIFFICULTY': 'Beginner',
        'METER': 1,
        'RADARVALUES': '0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0,0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0',
        'BPMS': '0=184.57'
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(step_info[key])), file=output_stepfile)

In [87]:
def write_notes(output_stepfile):
    print ("#NOTES:", file=output_stepfile)
    
    for i in range(5):
        print ("0101\n,", file=output_stepfile)
    print ("0000;", file=output_stepfile)

In [89]:
output_stepfile=open('StepMania/Songs/StepMania 5/{0}/{0}.ssc'.format(song_name), 'w')
from __future__ import print_function
write_song_header(output_stepfile)
write_step_header(output_stepfile)
write_notes(output_stepfile)
output_stepfile.close()